In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-4-4996ee3d8d09>", line 2, in <module>
    drive.mount('/content/gdrive')
  File "/usr/local/lib/python3.7/dist-packages/google/colab/drive.py", line 115, in mount
    ephemeral=True)
  File "/usr/local/lib/python3.7/dist-packages/google/colab/drive.py", line 135, in _mount
    'request_auth', request={'authType': 'dfs_ephemeral'}, timeout_sec=None)
  File "/usr/local/lib/python3.7/dist-packages/google/colab/_message.py", line 175, in blocking_request
    return read_reply_from_input(request_id, timeout_sec)
  File "/usr/local/lib/python3.7/dist-packages/google/colab/_message.py", line 101, in read_reply_from_input
    time.sleep(0.025)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/loc

KeyboardInterrupt: ignored

In [3]:
# from google.colab import files
# files.upload()
!cp -r /content/gdrive/MyDrive/NCT-CRC /content

^C


## 先生成训练集和测试集的索引文件

In [ ]:
import pandas as pd
import cv2
import torchvision
import matplotlib.pyplot as plt
import numpy as np
import os
from sklearn.utils import shuffle
Imgfolder = "/content/NCT-CRC"
image_ = []
label_ = []
tissues_name = os.listdir(Imgfolder)
for tis in tissues_name:
            tissues_dir = os.path.join(Imgfolder, tis)
            tissues_imgs = os.listdir(tissues_dir)
            for img in tissues_imgs:
                image_dir = os.path.join(tissues_dir,img)
                image_.append(image_dir)
                label_.append(tis)
ll = {"path":image_, "label":label_}            
df = pd.DataFrame(ll)
df = shuffle(df)
train = df[0:90000]
test = df[90001:100000]
# train.to_csv('/content/gdrive/MyDrive/tissue_segmentation/readdata/NCT_CRC_train.csv')
# test.to_csv('/content/gdrive/MyDrive/tissue_segmentation/readdata/NCT_CRC_test.csv')
train.to_csv('/content/tissue_segmentation/readdata/NCT_CRC_test.csv', encoding="utf-8-sig", mode="a")
test.to_csv('/content/tissue_segmentation/readdata/NCT_CRC_train.csv', encoding="utf-8-sig", mode="a")
print(train)

                                                    path label
22602  /content/gdrive/MyDrive/NCT-CRC/LYM/LYM-ERDFYC...   LYM
21688  /content/gdrive/MyDrive/NCT-CRC/LYM/LYM-RIPPHF...   LYM
65268  /content/gdrive/MyDrive/NCT-CRC/MUC/MUC-GPLQSP...   MUC
7077   /content/gdrive/MyDrive/NCT-CRC/BACK/BACK-ELLW...  BACK
77870  /content/gdrive/MyDrive/NCT-CRC/TUM/TUM-WSFQKT...   TUM
...                                                  ...   ...
6265   /content/gdrive/MyDrive/NCT-CRC/BACK/BACK-KLSN...  BACK
54886  /content/gdrive/MyDrive/NCT-CRC/ADI/ADI-RSFMRQ...   ADI
76820  /content/gdrive/MyDrive/NCT-CRC/TUM/TUM-VLARLS...   TUM
860    /content/gdrive/MyDrive/NCT-CRC/BACK/BACK-DDYC...  BACK
15795  /content/gdrive/MyDrive/NCT-CRC/DEB/DEB-QSMLAN...   DEB

[86452 rows x 2 columns]


In [ ]:
! nvidia-smi

Mounted at /content/gdrive


In [ ]:
import imp
import os
import numpy as np
# from sklearn import datasets
import torch
import torchvision
import argparse
from torch.utils.tensorboard import SummaryWriter
# path_to_module = "content/tissue_segmentation"
# sys.path.append(path_to_module)
from tissue_segmentation.utils import yaml_config_hook, save_model
from tissue_segmentation.modules import transform, resnet, network, contrastive_loss
from torch.utils import data
from tissue_segmentation.readdata import NCT_CRC_Data, dataloader
from tissue_segmentation.cluster import inference
from tissue_segmentation.evaluation.evaluation import evaluate
def NCT_CRC_train():
    loss_epoch = 0
    for step, (x_i, x_j, _) in enumerate(dataloader_train):
        optimizer.zero_grad()
        x_i = x_i.to('cuda')
        x_j = x_j.to('cuda')
        z_i, z_j, c_i, c_j = model(x_i, x_j)
        loss_instance = criterion_instance(z_i, z_j)
        loss_cluster = criterion_cluster(c_i, c_j)
        loss = loss_instance + loss_cluster
        loss.backward()
        optimizer.step()
        if step % 50 == 0:
            print(
                f"Step [{step}/{len(dataloader_train)}]\t loss_instance: {loss_instance.item()}\t loss_cluster: {loss_cluster.item()}")
        loss_epoch += loss.item()
    return loss_epoch

def NCT_CRC_test():
    print("start validate:")
    X, Y = inference(dataloader_tset, model, 'cuda')
    nmi, ari, f, acc = evaluate.evaluate(Y, X)
    print('NMI = {:.4f} ARI = {:.4f} F = {:.4f} ACC = {:.4f}'.format(nmi, ari, f, acc))
    


if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    config = yaml_config_hook("/content/tissue_segmentation/config/config.yaml")
    for k, v in config.items():
        parser.add_argument(f"--{k}", default=v, type=type(v))
    args = parser.parse_args(["--seed",'42',
"--workers",'2',
"--dataset_dir","./datasets",
"--batch_size",'512',
"--image_size",'224',
"--start_epoch",'0',
"--epochs",'1000',
"--dataset","CIFAR-10",
"--resnet","ResNet18",
"--feature_dim",'128',
"--model_path","/content/gdrive/MyDrive/tissue_segmentation/save",
"--reload",'False',
"--learning_rate",'0.0003',
"--weight_decay",'0.',
"--instance_temperature",'0.5',
"--cluster_temperature",'1.0',])
    if not os.path.exists(args.model_path):
        os.makedirs(args.model_path)

    torch.manual_seed(args.seed)
    torch.cuda.manual_seed_all(args.seed)
    torch.cuda.manual_seed(args.seed)
    np.random.seed(args.seed)
    print(f"batchsize:{args.batch_size}")
    # prepare data
    class_num = 9
    folder_path = '/content/NCT-CRC'
    dataset_train = NCT_CRC_Data('/content/gdrive/MyDrive/NCT-CRC')
    dataloader_train = torch.utils.data.DataLoader(
        dataset_train,
        batch_size=args.batch_size,
        shuffle=False,
        drop_last=True,
        num_workers=2,
    )
    dataset_test = NCT_CRC_Data(folder_path, train=False)
    dataloader_tset = torch.utils.data.DataLoader(
        dataset_test,
        batch_size=args.batch_size,
        shuffle=False,
        drop_last=True,
        num_workers = 2,
    )
    # initialize model
    writer = SummaryWriter()
    res = resnet.get_resnet(args.resnet)
    model = network.Network(res, args.feature_dim, class_num)
    model = model.to('cuda')
    # optimizer / loss
    optimizer = torch.optim.Adam(model.parameters(), lr=args.learning_rate, weight_decay=args.weight_decay)
    model_fp = '/content/tissue_segmentation/save/checkpoint_60.tar'
    checkpoint = torch.load(model_fp)
    model.load_state_dict(checkpoint['net'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    args.start_epoch = checkpoint['epoch'] + 1
    # if args.reload:
    #     model_fp = os.path.join(args.model_path, "checkpoint_{}.tar".format(args.start_epoch))
    #     checkpoint = torch.load(model_fp)
    #     model.load_state_dict(checkpoint['net'])
    #     optimizer.load_state_dict(checkpoint['optimizer'])
    #     args.start_epoch = checkpoint['epoch'] + 1
    loss_device = torch.device("cuda")
    criterion_instance = contrastive_loss.InstanceLoss(args.batch_size, args.instance_temperature, loss_device).to(
        loss_device)
    criterion_cluster = contrastive_loss.ClusterLoss(class_num, args.cluster_temperature, loss_device).to(loss_device)
    # train
    for epoch in range(args.start_epoch, args.epochs):
        lr = optimizer.param_groups[0]["lr"]
        loss_epoch = NCT_CRC_train()
        if epoch % 10 == 0:
            save_model(args, model, optimizer, epoch)
        writer.add_scalar('Loss/train', loss_epoch, epoch)
        print(f"Epoch [{epoch}/{args.epochs}]\t train_Loss: {loss_epoch / len(dataloader_train)}")
        # NCT_CRC_test()
        
    save_model(args, model, optimizer, args.epochs)


batchsize:512
Step [0/168]	 loss_instance: 6.173172950744629	 loss_cluster: 2.4927804470062256
Step [50/168]	 loss_instance: 5.9727559089660645	 loss_cluster: 2.3523452281951904
Step [100/168]	 loss_instance: 5.746988296508789	 loss_cluster: 2.263152599334717
Step [150/168]	 loss_instance: 5.777924060821533	 loss_cluster: 2.2825751304626465
Epoch [61/1000]	 train_Loss: 8.15124638591494
Step [0/168]	 loss_instance: 6.152624130249023	 loss_cluster: 2.459198236465454
Step [50/168]	 loss_instance: 5.978278636932373	 loss_cluster: 2.3448379039764404
Step [100/168]	 loss_instance: 5.745514392852783	 loss_cluster: 2.2780184745788574
Step [150/168]	 loss_instance: 5.799169063568115	 loss_cluster: 2.28631854057312
Epoch [62/1000]	 train_Loss: 8.14192742676962
Step [0/168]	 loss_instance: 6.131110668182373	 loss_cluster: 2.487013578414917
Step [50/168]	 loss_instance: 6.018303394317627	 loss_cluster: 2.374803066253662
Step [100/168]	 loss_instance: 5.715963363647461	 loss_cluster: 2.253080368041

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
! pip install munkres